In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.width', 1000)  # Change 1000 to the desired width
pd.set_option('display.max_columns', None)  # Show all columns


# Load the dataset
file_path = "slot_game_data.csv"
df = pd.read_csv(file_path)

# Normalize the features
scaler = MinMaxScaler()
df[['session_duration_hours', 'money_spent', 'avg_bet_amount']] = scaler.fit_transform(df[['session_duration_hours', 'money_spent', 'avg_bet_amount']])

# Define weights (you can adjust these based on domain knowledge)
weights = {
    'session_duration_hours': 0.4,
    'money_spent': 0.4,
    'avg_bet_amount': 0.2
}

# Calculate the score
df['score'] = (df['session_duration_hours'] * weights['session_duration_hours'] +
               df['money_spent'] * weights['money_spent'] +
               df['avg_bet_amount'] * weights['avg_bet_amount'])

# Aggregate scores by player_id and game_id (e.g., using mean)
df = df.groupby(['player_id', 'game_id'])['score'].mean().reset_index()

# Create a mapping of unique player and game IDs to indices
player_mapping = {player: idx for idx, player in enumerate(df['player_id'].unique())}
game_mapping = {game: idx for idx, game in enumerate(df['game_id'].unique())}

df['player_id'] = df['player_id'].map(player_mapping)
df['game_id'] = df['game_id'].map(game_mapping)

# Check if the aggregation was successful
print(df.head())

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2)


   player_id  game_id     score
0          0        0  0.298322
1          0        1  0.465837
2          0        2  0.506277
3          0        3  0.341687
4          0        4  0.667351


In [2]:
class NFM(nn.Module):
    def __init__(self, num_users, num_games, embedding_dim=50, hidden_layers=[128, 64]):
        super(NFM, self).__init__()
        
        # User and Game embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.game_embedding = nn.Embedding(num_games, embedding_dim)
        
        # MLP layers (hidden layers)
        layers = []
        in_size = embedding_dim * 3  # user + game + interaction term
        for out_size in hidden_layers:
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            in_size = out_size
        
        self.fc_layers = nn.Sequential(*layers)
        
        # Output layer (single output)
        self.output = nn.Linear(hidden_layers[-1], 1)
        
        # Bias terms for user and game
        self.user_bias = nn.Embedding(num_users, 1)
        self.game_bias = nn.Embedding(num_games, 1)

    def forward(self, user, game):
        # Get embeddings
        user_emb = self.user_embedding(user)
        game_emb = self.game_embedding(game)
        
        # Interaction term (FM part): pairwise interaction between user and game
        interaction = user_emb * game_emb  # element-wise multiplication
        
        # Concatenate user, game, and interaction features
        x = torch.cat([user_emb, game_emb, interaction], dim=1)
        
        # Pass through MLP layers
        x = self.fc_layers(x)
        
        # Output prediction
        x = self.output(x)
        
        # Add biases for user and game, but now we will properly broadcast them
        user_b = self.user_bias(user).squeeze()  # Shape [batch_size, 1] to [batch_size]
        game_b = self.game_bias(game).squeeze()  # Shape [batch_size, 1] to [batch_size]
        
        # Add the bias terms
        x += user_b + game_b
        
        return x.squeeze()  # Output shape: [batch_size]


In [ ]:
# Function to Predict Scores for a Batch of Games
def predict_scores_batch(user_id, games, model, device="cuda"):
    """
    Predict engagement scores for a given user and list of games in batches.
    """
    user_tensor = torch.tensor([user_id] * len(games), dtype=torch.long).to(device)
    game_tensor = torch.tensor(games, dtype=torch.long).to(device)

    with torch.no_grad():
        predictions = model(user_tensor, game_tensor)
    
    scores = predictions.cpu().numpy().flatten()
    return dict(zip(games, scores))

# Function to Recommend Top-N Games
def recommend_top_n(user_id, games, model, df, n=5, recommend_new_only=True, device="cuda"):
    """
    Recommend the top-N slot games for a single user.
    """
    if recommend_new_only:
        # Optimize: Only consider unplayed games
        played_games = set(df[df['player_id'] == user_id]['game_id'].unique())
        games_to_recommend = [game_id for game_id in games if game_id not in played_games]
    else:
        # Consider both played and unplayed games
        games_to_recommend = games
    
    # Predict scores for the selected games in a batch
    scores = predict_scores_batch(user_id, games_to_recommend, model, device)
    
    # Sort games based on predicted scores
    sorted_games = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Return the top-N recommended game IDs
    return [game_id for game_id, _ in sorted_games[:n]]

# Get all game IDs
all_game_ids = df["game_id"].unique().tolist()

# Test for an existing user
existing_user_id = 123  # Change to an actual user ID

# Option 1: Recommend only new slots
print("Recommendations for new slots only:", recommend_top_n(existing_user_id, all_game_ids, model, df, recommend_new_only=True))

# Option 2: Recommend all slots (including played ones)
print("Recommendations for all slots:", recommend_top_n(existing_user_id, all_game_ids, model, df, recommend_new_only=False))
